### Rental Listing (Word2vec 활용) 최종 발표 - by 이상열

In [1]:
#라이브러리 로딩
import os
import sys
import operator
import numpy as np
import pandas as pd
from scipy import sparse
import xgboost as xgb
import random
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
sns.set(style='ticks')
color = sns.color_palette()

import thinkstats2, thinkplot # Do "pip install thinkx", if necessary

%matplotlib inline

#Ref : https://github.com/KaggleBreak/problems/blob/master/2015_06_bagofwords/bagofwords.ipynb 

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import nltk
#nltk.download()
from nltk.corpus import stopwords

In [3]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words    
    # The input is a single string (a raw movie review), and  he output is a single string (a preprocessed movie review)    
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text()     
    #    
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)     
    #    
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                                 
    #   
    # 4. In Python, searching a set is much faster than searching    
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                      
    #     
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]       
    #    
    # 6. Join the words back into one string separated by space,     
    # and return the result.
    return( " ".join( meaningful_words ))

In [4]:
#input data
train_df=pd.read_json('./data/train.json')
test_df=pd.read_json('./data/test.json')

#basic features
train_df["price_t"] =train_df["price"]/train_df["bedrooms"]
test_df["price_t"] = test_df["price"]/test_df["bedrooms"] 
train_df["room_sum"] = train_df["bedrooms"]+train_df["bathrooms"] 
test_df["room_sum"] = test_df["bedrooms"]+test_df["bathrooms"] 

# count of photos #
train_df["num_photos"] = train_df["photos"].apply(len)
test_df["num_photos"] = test_df["photos"].apply(len)

# count of "features" #
train_df["num_features"] = train_df["features"].apply(len)
test_df["num_features"] = test_df["features"].apply(len)

# count of words present in description column #
train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))
test_df["num_description_words"] = test_df["description"].apply(lambda x: len(x.split(" ")))


# 원래 있던 original값들과 새로만든 feature를 넣어 feature_to_use 리스트 만듬
features_to_use=["bathrooms", "bedrooms", "latitude", "longitude", "price","price_t","num_photos", "num_features", "num_description_words","listing_id"]


In [5]:
# Get the number of reviews based on the dataframe column size
num_reviews = train_df["description"].size

In [6]:
num_reviews

49352

In [7]:
clean_train_reviews = []

In [28]:
train_df["description"].iloc[0]

"A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy These Following Apartment Features As You Rent Here? Modern Designed Bathroom w/ a Deep Spa Soaking Tub? Room to Room AC/Heat? Real Oak Hardwood Floors? Rain Forest Shower Head? SS steel Appliances w/ Chef Gas Cook Oven & LG Fridge? washer /dryer in the apt? Cable Internet Ready? Granite Counter Top Kitchen w/ lot of cabinet storage spaceIt's Just A Few blocks To L Train<br /><br />Don't miss out!<br /><br />We have several great apartments in the immediate area.<br /><br />For additional information 687-878-2229<p><a  website_redacted "

In [32]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [35]:
for i in range( 0, num_reviews ):
    if( (i+1)%1000 == 0 ):
        print ("Review %d of %d\n" % ( i+1, num_reviews ))
    clean_train_reviews.append( review_to_words( train_df["description"].iloc[i] ) )

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


Review 1000 of 49352

Review 2000 of 49352

Review 3000 of 49352

Review 4000 of 49352

Review 5000 of 49352

Review 6000 of 49352

Review 7000 of 49352

Review 8000 of 49352

Review 9000 of 49352

Review 10000 of 49352

Review 11000 of 49352

Review 12000 of 49352

Review 13000 of 49352

Review 14000 of 49352

Review 15000 of 49352

Review 16000 of 49352

Review 17000 of 49352

Review 18000 of 49352

Review 19000 of 49352

Review 20000 of 49352

Review 21000 of 49352

Review 22000 of 49352

Review 23000 of 49352

Review 24000 of 49352

Review 25000 of 49352

Review 26000 of 49352

Review 27000 of 49352

Review 28000 of 49352

Review 29000 of 49352

Review 30000 of 49352

Review 31000 of 49352

Review 32000 of 49352

Review 33000 of 49352

Review 34000 of 49352

Review 35000 of 49352

Review 36000 of 49352



/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


Review 37000 of 49352

Review 38000 of 49352

Review 39000 of 49352

Review 40000 of 49352

Review 41000 of 49352

Review 42000 of 49352

Review 43000 of 49352

Review 44000 of 49352

Review 45000 of 49352

Review 46000 of 49352

Review 47000 of 49352

Review 48000 of 49352

Review 49000 of 49352



In [37]:
clean_train_reviews[2]

'top top west village location beautiful pre war building laundry basement live super apartment features large bedroom closet separate living room kitchen features granite tops dishwasher microwave included marble bathroom hardwood flooring building well maintained conveniently located near c e l trains surrounded many local cafe restaurants available november st move view apartment please contact via email call number listed bond new york real estate broker supports equal housing opportunity'

In [38]:
def review_to_wordlist( review, remove_stopwords=False ):    
    # Function to convert a document to a sequence of words,    
    # optionally removing stop words.  Returns a list of words.    
    #    
    # 1. Remove HTML    
    review_text = BeautifulSoup(review).get_text()    
    #      
    # 2. Remove non-letters    
    review_text = re.sub("[^a-zA-Z]"," ", review_text)    
    #    
    # 3. Convert words to lower case and split them    
    words = review_text.lower().split()    
    #    
    # 4. Optionally remove stop words (false by default)    
    if remove_stopwords:        
        stops = set(stopwords.words("english"))        
        words = [w for w in words if not w in stops]    
        #    
        # 5. Return a list of words    
    return(words)

In [39]:
def review_to_sentences( review, tokenizer, remove_stopwords=False ):    
    # Function to split a review into parsed sentences. Returns a     
    # list of sentences, where each sentence is a list of words    
    #    
    # 1. Use the NLTK tokenizer to split the paragraph into sentences    
    raw_sentences = tokenizer.tokenize(review.strip())
    
    #    
    # 2. Loop over each sentence    
    sentences = []    
    for raw_sentence in raw_sentences:        
        # If a sentence is empty, skip it        
        if len(raw_sentence) > 0:            
            # Otherwise, call review_to_wordlist to get a list of words            
            sentences.append( review_to_wordlist( raw_sentence, remove_stopwords ))    
            #    
            # Return the list of sentences (each sentence is a list of words,    
            # so this returns a list of lists    
    return sentences

In [40]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
#http://dev-strender.github.io/text%20processing/2015/04/05/Text-Processing-1/

In [45]:
tokenizer.tokenize(train_df["description"].iloc[0].strip())[0]

'A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy These Following Apartment Features As You Rent Here?'

In [47]:
tokenizer.tokenize(train_df["description"].iloc[0].strip())[1]

'Modern Designed Bathroom w/ a Deep Spa Soaking Tub?'

In [49]:
review_to_wordlist(tokenizer.tokenize(train_df["description"].iloc[0].strip())[1],False)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


['modern', 'designed', 'bathroom', 'w', 'a', 'deep', 'spa', 'soaking', 'tub']

In [50]:
sentences = []  # Initialize an empty list of sentences

In [52]:
print("Parsing sentences from training set")
for review in train_df["description"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in test_df["description"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


Parsing sentences from unlabeled set


In [54]:
print(len(sentences))

583404


In [56]:
print(sentences[0])

['a', 'brand', 'new', 'bedroom', 'bath', 'apartmentenjoy', 'these', 'following', 'apartment', 'features', 'as', 'you', 'rent', 'here']


In [57]:
print(sentences[1])

['modern', 'designed', 'bathroom', 'w', 'a', 'deep', 'spa', 'soaking', 'tub']


In [59]:
from gensim.models import word2vec
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [97]:
# Initialize and train the model (this will take some time)from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)
# If you don't plan to train the model any further, calling # init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)
# It can be helpful to create a meaningful model name and # save the model for later use. You can load it later using Word2Vec.load()
model_name = "data/300features_40minwords_10context"
model.save(model_name)

Training model...


In [61]:
model.doesnt_match("man woman child kitchen".split())

'child'

In [64]:
model.most_similar("room")

[('rooms', 0.5115649700164795),
 ('area', 0.504125714302063),
 ('facility', 0.4711500406265259),
 ('quarters', 0.3973737359046936),
 ('den', 0.3888862431049347),
 ('facilities', 0.3870880603790283),
 ('roomcall', 0.3670872151851654),
 ('space', 0.36468058824539185),
 ('table', 0.3554668128490448),
 ('plus', 0.3495696783065796)]

In [65]:
model = word2vec.Word2Vec.load("data/300features_40minwords_10context")

In [66]:
model.syn0.shape

(5507, 300)

In [67]:
model["building"]

array([ 0.03375647,  0.01310883,  0.10571329, -0.07222398, -0.02576954,
        0.02231651, -0.13409632,  0.05058629, -0.03423413,  0.00734794,
        0.08761748,  0.06157919,  0.12037229,  0.02186078,  0.02035874,
       -0.10238677, -0.07350989, -0.01296252, -0.01660722,  0.011027  ,
       -0.00940361, -0.02979136,  0.08320071,  0.07187173,  0.08391818,
       -0.0088168 ,  0.00340236, -0.01581809, -0.07416918, -0.0225496 ,
       -0.01325632,  0.04519134,  0.02077758,  0.05758807, -0.03345492,
       -0.05530149,  0.01606602,  0.01502326,  0.05069974,  0.04435149,
       -0.01590603,  0.02161378,  0.06297158, -0.17924659,  0.00894811,
        0.04633858, -0.08254109,  0.0657755 , -0.03758929,  0.04434289,
       -0.0192487 ,  0.00477618,  0.1101819 , -0.04443537, -0.08881859,
       -0.08462416, -0.09394614,  0.07577276, -0.05096463, -0.02676815,
        0.02600441, -0.02582833,  0.01318391,  0.08707616, -0.0316474 ,
       -0.01451613,  0.02299187,  0.04715926,  0.05699749, -0.04

In [68]:
#From Words To Paragraphs, Attempt 1: Vector Averaging¶
import numpy as np  
# Make sure that numpy is imported
def makeFeatureVec(words, model, num_features):    
    # Function to average all of the word vectors in a given    
    # paragraph    
    #    
    # Pre-initialize an empty numpy array (for speed)    
    featureVec = np.zeros((num_features,),dtype="float32")    
    #    
    nwords = 0.    
    #     
    # Index2word is a list that contains the names of the words in     
    # the model's vocabulary. Convert it to a set, for speed     
    index2word_set = set(model.index2word)    
    #    
    # Loop over each word in the review and, if it is in the model's    
    # vocaublary, add its feature vector to the total    
    for word in words:        
        if word in index2word_set:             
            nwords = nwords + 1.            
            featureVec = np.add(featureVec,model[word])    
    #     
    # Divide the result by the number of words to get the average    
    featureVec = np.divide(featureVec,nwords)    
    return featureVec

def getAvgFeatureVecs(reviews, model, num_features):    
    # Given a set of reviews (each one a list of words), calculate     
    # the average feature vector for each one and return a 2D numpy array     
    #     
    # Initialize a counter    
    counter = 0.    
    #     
    # Preallocate a 2D numpy array, for speed    
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")    
    #     
    # Loop through the reviews    
    for review in reviews:       
        #       
        # Print a status message every 1000th review       
        if counter%1000. == 0.:           
            print ("Review %d of %d" % (counter, len(reviews)))      
            #        
            # Call the function (defined above) that makes average feature vectors       
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)       
        #       
        # Increment the counter       
        counter = counter + 1.    
    return reviewFeatureVecs

In [69]:
clean_train_reviews = []

In [70]:
for review in train_df["description"]:    
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))
trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


Review 0 of 49352


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:25: RuntimeWarning: invalid value encountered in true_divide


Review 1000 of 49352
Review 2000 of 49352
Review 3000 of 49352
Review 4000 of 49352
Review 5000 of 49352
Review 6000 of 49352
Review 7000 of 49352
Review 8000 of 49352
Review 9000 of 49352
Review 10000 of 49352
Review 11000 of 49352
Review 12000 of 49352
Review 13000 of 49352
Review 14000 of 49352
Review 15000 of 49352
Review 16000 of 49352
Review 17000 of 49352
Review 18000 of 49352
Review 19000 of 49352
Review 20000 of 49352
Review 21000 of 49352
Review 22000 of 49352
Review 23000 of 49352
Review 24000 of 49352
Review 25000 of 49352
Review 26000 of 49352
Review 27000 of 49352
Review 28000 of 49352
Review 29000 of 49352
Review 30000 of 49352
Review 31000 of 49352
Review 32000 of 49352
Review 33000 of 49352
Review 34000 of 49352
Review 35000 of 49352
Review 36000 of 49352
Review 37000 of 49352
Review 38000 of 49352
Review 39000 of 49352
Review 40000 of 49352
Review 41000 of 49352
Review 42000 of 49352
Review 43000 of 49352
Review 44000 of 49352
Review 45000 of 49352
Review 46000 of 493

In [71]:
trainDataVecs

array([[ 0.00055333, -0.0183891 ,  0.00794306, ...,  0.01932015,
         0.02837954, -0.00214033],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [ 0.0116992 , -0.01275395,  0.0106168 , ..., -0.00145144,
         0.00130717,  0.00064602],
       ..., 
       [ 0.02533396, -0.02877892,  0.01583145, ...,  0.0025978 ,
         0.00494621, -0.00066494],
       [-0.02255059, -0.00497683,  0.01731538, ...,  0.01652828,
         0.01012026, -0.00421657],
       [ 0.00916268, -0.01646173,  0.00261846, ...,  0.00139349,
         0.00044382,  0.00409133]], dtype=float32)

In [72]:
print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test_df["description"]:    
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))
testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Creating average feature vecs for test reviews


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


Review 0 of 74659


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:25: RuntimeWarning: invalid value encountered in true_divide


Review 1000 of 74659
Review 2000 of 74659
Review 3000 of 74659
Review 4000 of 74659
Review 5000 of 74659
Review 6000 of 74659
Review 7000 of 74659
Review 8000 of 74659
Review 9000 of 74659
Review 10000 of 74659
Review 11000 of 74659
Review 12000 of 74659
Review 13000 of 74659
Review 14000 of 74659
Review 15000 of 74659
Review 16000 of 74659
Review 17000 of 74659
Review 18000 of 74659
Review 19000 of 74659
Review 20000 of 74659
Review 21000 of 74659
Review 22000 of 74659
Review 23000 of 74659
Review 24000 of 74659
Review 25000 of 74659
Review 26000 of 74659
Review 27000 of 74659
Review 28000 of 74659
Review 29000 of 74659
Review 30000 of 74659
Review 31000 of 74659
Review 32000 of 74659
Review 33000 of 74659
Review 34000 of 74659
Review 35000 of 74659
Review 36000 of 74659
Review 37000 of 74659
Review 38000 of 74659
Review 39000 of 74659
Review 40000 of 74659
Review 41000 of 74659
Review 42000 of 74659
Review 43000 of 74659
Review 44000 of 74659
Review 45000 of 74659
Review 46000 of 746

In [73]:
testDataVecs

array([[ 0.0138732 , -0.02027832, -0.01282191, ...,  0.00305912,
        -0.00083949,  0.00687549],
       [ 0.01392211, -0.01963464,  0.00797748, ...,  0.01133472,
         0.00225028,  0.0175917 ],
       [-0.00418794, -0.00298554,  0.00774313, ...,  0.0045233 ,
         0.01313456,  0.00084954],
       ..., 
       [-0.01202475, -0.02804098, -0.00189287, ...,  0.00702829,
         0.02023262, -0.00015413],
       [ 0.00467204, -0.00276794,  0.01279011, ..., -0.00600866,
        -0.00233   , -0.00191295],
       [ 0.01138543, -0.02832986,  0.00448089, ...,  0.00433501,
         0.01560072,  0.0004311 ]], dtype=float32)

In [74]:
trainDataVecs.shape

(49352, 300)

In [76]:
testDataVecs.shape

(74659, 300)

In [77]:
#https://flonelin.wordpress.com/2016/07/26/tuning-xgboostextream-gradient-boosting/
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0, num_rounds=1000):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.03
    param['max_depth'] = 6
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7 # 70프로만 뽑겠다
    param['colsample_bytree'] = 0.7 # 컬럼의 ratio 최대 몇개만 쓰겠다라는 의미?
    param['seed'] = seed_val # 초기값 설정 랜덤 안되게
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y) # D매트릭스로 바꿔줘야 한다.

    # test의 타겟값을 넣은 xgb model
    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    # test의 타겟값을 넣지 않은 xgb model
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

In [78]:
# train data 의 수만큼의 list를 만듬
index=list(range(train_df.shape[0]))
random.shuffle(index) # 값 셔플
a=[np.nan]*len(train_df) # 데이터 수만큼 리스트 만들고
b=[np.nan]*len(train_df)
c=[np.nan]*len(train_df)

In [79]:
# 데이터를 5등분할 계획
# 매니저id를 key로 하는 building_level을 만듦
for i in range(5): 
    building_level={}
    for j in train_df['manager_id'].values:
        building_level[j]=[0,0,0]
    test_index=index[int((i*train_df.shape[0])/5):int(((i+1)*train_df.shape[0])/5)]
    train_index=list(set(index).difference(test_index))
    for j in train_index:
        temp=train_df.iloc[j]
        if temp['interest_level']=='low':
            building_level[temp['manager_id']][0]+=1
        if temp['interest_level']=='medium':
            building_level[temp['manager_id']][1]+=1
        if temp['interest_level']=='high':
            building_level[temp['manager_id']][2]+=1
    for j in test_index:
        temp=train_df.iloc[j]
        if sum(building_level[temp['manager_id']])!=0:
            a[j]=building_level[temp['manager_id']][0]*1.0/sum(building_level[temp['manager_id']])
            b[j]=building_level[temp['manager_id']][1]*1.0/sum(building_level[temp['manager_id']])
            c[j]=building_level[temp['manager_id']][2]*1.0/sum(building_level[temp['manager_id']])
train_df['manager_level_low']=a
train_df['manager_level_medium']=b
train_df['manager_level_high']=c

In [90]:
# 이번엔 train데이터를 통해 집계한 매니저에 대한 'interest level별 count정보'로 test데이터의 'manager에 대한 interest level별 비율정보'을 구함
a=[]
b=[]
c=[]
building_level={}
for j in train_df['manager_id'].values:
    building_level[j]=[0,0,0]
for j in range(train_df.shape[0]):
    temp=train_df.iloc[j]
    if temp['interest_level']=='low':
        building_level[temp['manager_id']][0]+=1
    if temp['interest_level']=='medium':
        building_level[temp['manager_id']][1]+=1
    if temp['interest_level']=='high':
        building_level[temp['manager_id']][2]+=1

for i in test_df['manager_id'].values:
    if i not in building_level.keys():
        a.append(np.nan)
        b.append(np.nan)
        c.append(np.nan)
    else:
        a.append(building_level[i][0]*1.0/sum(building_level[i]))
        b.append(building_level[i][1]*1.0/sum(building_level[i]))
        c.append(building_level[i][2]*1.0/sum(building_level[i]))
test_df['manager_level_low']=a
test_df['manager_level_medium']=b
test_df['manager_level_high']=c

In [80]:
#위에서 애써만든 매니저 평가정보를 여기에서 feature에 포함시킴
features_to_use.append('manager_level_low') 
features_to_use.append('manager_level_medium') 
features_to_use.append('manager_level_high')

In [91]:
# 범주형 데이터를 넣는 과정
categorical = ["display_address", "manager_id", "building_id", "street_address"]
for f in categorical:
        if train_df[f].dtype=='object':
            #print(f)
            lbl = preprocessing.LabelEncoder() #범주형 변수를 사용하기 위해 팩터화시키는것
            lbl.fit(list(train_df[f].values) + list(test_df[f].values))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))
            features_to_use.append(f)

In [82]:
# original데이터의 features컬럼을 tfidf 팩터화
train_df['features'] = train_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
test_df['features'] = test_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
print(train_df["features"].head())

10                                                         
10000     Doorman Elevator Fitness_Center Cats_Allowed D...
100004    Laundry_In_Building Dishwasher Hardwood_Floors...
100007                               Hardwood_Floors No_Fee
100013                                              Pre-War
Name: features, dtype: object


In [83]:
# original데이터의 features컬럼을 tfidf 팩터화
tfidf = CountVectorizer(stop_words='english', max_features=200)
tr_sparse = tfidf.fit_transform(train_df["features"])
te_sparse = tfidf.transform(test_df["features"])

In [89]:
features_to_use

['bathrooms',
 'bedrooms',
 'latitude',
 'longitude',
 'price',
 'price_t',
 'num_photos',
 'num_features',
 'num_description_words',
 'listing_id',
 'manager_level_low',
 'manager_level_medium',
 'manager_level_high',
 'display_address',
 'manager_id',
 'building_id',
 'street_address']

In [86]:
type(trainDataVecs)

scipy.sparse.csr.csr_matrix

In [92]:
# xgb에 넣기위해 자료구조를 만듬
train_X = sparse.hstack([train_df[features_to_use], tr_sparse, trainDataVecs]).tocsr()
test_X = sparse.hstack([test_df[features_to_use], te_sparse, testDataVecs]).tocsr()

In [93]:
target_num_map = {'high':0, 'medium':1, 'low':2}
train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))
print(train_X.shape, test_X.shape)

(49352, 517) (74659, 517)


In [94]:
cv_scores = []
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
for dev_index, val_index in kf.split(range(train_X.shape[0])):
        dev_X, val_X = train_X[dev_index,:], train_X[val_index,:]
        dev_y, val_y = train_y[dev_index], train_y[val_index]
        preds, model = runXGB(dev_X, dev_y, val_X, val_y)
        cv_scores.append(log_loss(val_y, preds))
        print(cv_scores)
        break

[0]	train-mlogloss:1.07956	test-mlogloss:1.08009
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:1.06158	test-mlogloss:1.06264
[2]	train-mlogloss:1.04326	test-mlogloss:1.04477
[3]	train-mlogloss:1.02594	test-mlogloss:1.02795
[4]	train-mlogloss:1.00978	test-mlogloss:1.0123
[5]	train-mlogloss:0.993963	test-mlogloss:0.997064
[6]	train-mlogloss:0.978094	test-mlogloss:0.981778
[7]	train-mlogloss:0.963401	test-mlogloss:0.967504
[8]	train-mlogloss:0.949002	test-mlogloss:0.953572
[9]	train-mlogloss:0.935136	test-mlogloss:0.940238
[10]	train-mlogloss:0.922337	test-mlogloss:0.927935
[11]	train-mlogloss:0.910074	test-mlogloss:0.916082
[12]	train-mlogloss:0.898003	test-mlogloss:0.904555
[13]	train-mlogloss:0.886384	test-mlogloss:0.893302
[14]	train-mlogloss:0.875145	test-mlogloss:0.882631
[15]	train-mlogloss:0.864804	test-mlogloss:0.8728
[16]	train-mlogloss:0.85447	test-mloglos

In [95]:
cv_scores

[0.54894200051355269]

In [96]:
preds, model = runXGB(train_X, train_y, test_X, num_rounds=1000)
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_csv("xgb_starter_word2vec.csv", index=False)

<img src="previos.png" width="500" height="200"/>
<img src="word2vec.png" width="500" height="200"/>